In [1]:
%reset -fs

In [2]:
import pandas as pd
import numpy as np
import time, os
from bs4 import BeautifulSoup
import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from dotenv import load_dotenv
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [3]:
load_dotenv()

True

In [4]:
mongo_pw = os.environ.get("MONGO_PW")

In [5]:
client = MongoClient('mongodb+srv://natedirenzo:' + mongo_pw + '@nba-db.grcwg.mongodb.net/myFirstDatabase?retryWrites=true&w=majority')


In [6]:
base = datetime.datetime.today()
delta = base - datetime.datetime(2021, 10, 19)
date_list = [base - datetime.timedelta(days=x) for x in range(delta.days+1)]
date_list = [d.strftime('%Y-%m-%d') for d in date_list]

In [7]:
#run this initially; then don't run again if you just want to keep adding to it
final_df = pd.DataFrame()

In [8]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/?date='

In [9]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver

In [10]:
driver = webdriver.Chrome(chromedriver)

In [11]:
# Tip of the hat to Rita for helping with these two functions !
def get_site_rb(date):
    bet_query = bet_search + date
    driver.get(bet_query)
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
    except:
        pass
    get_data_rb()


In [12]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 7:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = pd.concat([final_df, pd.DataFrame(data)])
    time.sleep(1)

    try:
        page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
        if int(page_progress[5]) == int(page_progress[10]):
            return final_df
        else:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            get_data_rb()
    except:
        return final_df


In [13]:
#run this and then close the cookies pop-up!
for i in date_list:
    
    if i == '2021-12-24':
        continue
    if i == '2021-11-25':
        continue
    if i == '2022-02-18':
        continue
    if i == '2022-02-19':
        continue
    if i == '2022-02-20':
        continue
    if i == '2022-02-21':
        continue
    if i == '2022-02-22':
        continue
    if i == '2022-02-23':
        continue
    get_site_rb(i)

In [14]:
driver.close()

In [15]:
#Clean up below
final_df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Pick', 'Moneyline', 'Result', 'Date', 'Accuracy']

In [16]:
final_df.reset_index(drop=True, inplace=True)

In [17]:
final_df

,Name,Team,Opp,Type,Line,Pick,Moneyline,Result,Date,Accuracy
0,1 R. RJ Barrett,NYK - F,vs. CLE,Points,26.50 (-125)21.00Diff: -5.50 Line: 26.5...,26.5,-125 TBD,TBD,04-02-2022,Last 7 days Top 15 Accuracy: 42-61 (41%) Sea...
1,2 . Bogdan Bogdanovic,ATL - G,vs. BKN,Points,16.50 (-115)11.40Diff: -5.10 Line: 16.5...,16.5,-115 TBD,TBD,04-02-2022,Last 7 days Top 15 Accuracy: 42-61 (41%) Sea...
2,3 O. Obi Toppin,NYK - F,vs. CLE,Points,16.50 (-115)11.60Diff: -4.90 Line: 16.5...,16.5,-115 TBD,TBD,04-02-2022,Last 7 days Top 15 Accuracy: 42-61 (41%) Sea...
3,4 I. Immanuel Quickley,NYK - G,vs. CLE,Points,16.50 (-111)12.20Diff: -4.30 Line: 16.5...,16.5,-111 TBD,TBD,04-02-2022,Last 7 days Top 15 Accuracy: 42-61 (41%) Sea...
4,5 K. Kevin Love,CLE - F,@ NYK,Points,10.50 (-128)13.60Diff: 3.10 Line: 10.50...,10.5,-128 TBD,TBD,04-02-2022,Last 7 days Top 15 Accuracy: 42-61 (41%) Sea...
...,...,...,...,...,...,...,...,...,...,...
14622,22 G. Grayson Allen,MIL - G,vs. BKN,Points,10.50 (-113)11.23Diff: 0.73 Line: 10.50...,10.5,-113,10 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1156-1128 (5...
14623,23 N. Nemanja Bjelica,GSW - F,@ LAL,Points,6.50 (-120)6.87Diff: 0.37 Line: 6.50 Co...,6.5,-120,15 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1156-1128 (5...
14624,24 D. DeAndre Jordan,LAL - C,vs. GSW,Points,4.50 (-130)4.82Diff: 0.32 Line: 4.50 Co...,4.5,-130,2 (under),10-19-2021,Season-to-date Top 15 Accuracy: 1156-1128 (5...
14625,25 P. Pat Connaughton,MIL - G,vs. BKN,Points,7.50 (-111)7.19Diff: -0.31 Line: 7.50 C...,7.5,-111,20 (over),10-19-2021,Season-to-date Top 15 Accuracy: 1156-1128 (5...


In [18]:
player_name = final_df.Name.str.split(" ", expand = True)
final_df['Name'] = player_name[2] + ' ' + player_name[4]

In [19]:
team = final_df.Team.str.split(" ", expand = True)
final_df['Team'] = team[0]
final_df['Position'] = team[2]

In [20]:
opp = final_df.Opp.str.split(" ", expand = True)
final_df.Opp = opp[1]
final_df['Home/Away'] = opp[0]

In [21]:
bet_info = final_df.Line.str.split(" ", expand = True)

In [22]:
money_line = final_df.Moneyline.str.split(" ", expand = True);money_line

,0,1,2,3,4,5,6,7
0,,,-125,,,,,TBD
1,,,-115,,,,,TBD
2,,,-115,,,,,TBD
3,,,-111,,,,,TBD
4,,,-128,,,,,TBD
...,...,...,...,...,...,...,...,...
14622,,,-113,None,None,None,None,None
14623,,,-120,None,None,None,None,None
14624,,,-130,None,None,None,None,None
14625,,,-111,None,None,None,None,None


In [27]:
money_line[3].value_counts()

    51
Name: 3, dtype: int64

In [31]:
for i in money_line[3]:
    for j in final_df['Result']:
        if i == None:
            continue
        elif i.isdigit() == True and j == 'TBD':
            j == i
        else:
            continue

In [32]:
final_df['Line'] = bet_info[7]
final_df['Projection'] = bet_info[15]
final_df['Diff'] = bet_info[9]
final_df['Moneyline'] = bet_info[13]
if len(bet_info[bet_info[16] == '']) > 0:
    final_df['over_under'] = bet_info[19]
else:
    final_df['over_under'] = bet_info[16]
final_df['over_under'] = final_df['over_under'].fillna('o')

In [33]:
final_df = final_df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy', 'over_under']]

In [34]:
final_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,Accuracy,over_under
0,04-02-2022,RJ Barrett,NYK,F,CLE,vs.,26.50,-125,21.00,-5.50,26.5,TBD,Last 7 days Top 15 Accuracy: 42-61 (41%) Sea...,u
1,04-02-2022,Bogdan,ATL,G,BKN,vs.,16.50,-115,11.40,-5.10,16.5,TBD,Last 7 days Top 15 Accuracy: 42-61 (41%) Sea...,u
2,04-02-2022,Obi Toppin,NYK,F,CLE,vs.,16.50,-115,11.60,-4.90,16.5,TBD,Last 7 days Top 15 Accuracy: 42-61 (41%) Sea...,u
3,04-02-2022,Immanuel Quickley,NYK,G,CLE,vs.,16.50,-111,12.20,-4.30,16.5,TBD,Last 7 days Top 15 Accuracy: 42-61 (41%) Sea...,u
4,04-02-2022,Kevin Love,CLE,F,NYK,@,10.50,-128,13.60o,3.10,10.5,TBD,Last 7 days Top 15 Accuracy: 42-61 (41%) Sea...,o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,10-19-2021,Grayson Allen,MIL,G,BKN,vs.,10.50,-113,11.23o,0.73,10.5,10 (under),Season-to-date Top 15 Accuracy: 1156-1128 (5...,o
14623,10-19-2021,Nemanja Bjelica,GSW,F,LAL,@,6.50,-120,6.87o,0.37,6.5,15 (over),Season-to-date Top 15 Accuracy: 1156-1128 (5...,o
14624,10-19-2021,DeAndre Jordan,LAL,C,GSW,vs.,4.50,-130,4.82o,0.32,4.5,2 (under),Season-to-date Top 15 Accuracy: 1156-1128 (5...,o
14625,10-19-2021,Pat Connaughton,MIL,G,BKN,vs.,7.50,-111,7.19,-0.31,7.5,20 (over),Season-to-date Top 15 Accuracy: 1156-1128 (5...,u


In [35]:
final_df = final_df.copy()

In [36]:
final_df['Projection'] = final_df['Projection'].map(lambda x: x.rstrip('ou'))

In [37]:
accuracies = final_df.Accuracy.str.split(" ", expand = True)

In [38]:
final_df['seven_day_accuracy'] = accuracies[8]

In [39]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [40]:
final_df.seven_day_accuracy = final_df.seven_day_accuracy.replace('', 50, regex=True)

In [41]:
final_df['seven_day_accuracy'] = pd.to_numeric(final_df['seven_day_accuracy'])

In [42]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x/100)

In [43]:
final_df.drop(columns=['Accuracy'], inplace=True)

In [44]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [45]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = final_df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)

In [46]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [47]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [48]:
sorted_df = sorted_df.replace('TBD', None)

In [49]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [51]:
sorted_df['Result'] = sorted_df.Result.apply(lambda x: str(x))

In [53]:
results = sorted_df.Result.str.split(" ", expand = True);results

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,11,,,,,,,,,(under)
1,,,,5,,,,,,,,,(under)
2,,,,14,,,,,,,,,(over)
3,,,,12,,,,,,,,,(under)
4,,,,14,,,,,,,,,(over)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,,,,16,,,,,,,,,(over)
14623,,,,13,,,,,,,,,(over)
14624,,,,11,,,,,,,,,(over)
14625,,,,0,,,,,,,,,(under)


In [62]:
sorted_df.Result = results[3]

In [63]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)
sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11.0,o,0.47
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5.0,o,0.45
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14.0,u,0.45
3,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12.0,o,0.45
4,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14.0,o,0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16.0,o,0.46
14623,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13.0,u,0.50
14624,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11.0,u,0.46
14625,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0.0,u,0.46


In [64]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    (sorted_df['Result'].isna() == True)
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0', np.nan]

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)

In [65]:
db = client['nba-data']

In [66]:
db.drop_collection('betting-data')

{'nIndexesWas': 1,
 'ns': 'nba-data.betting-data',
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1648947838, 23),
  'signature': {'hash': b'\xac`\xd8m\xe4x\xa3c\x9bo\xe4\x0fh\xca\xd0\xd0%,\x8dw',
   'keyId': 7018196322255634436}},
 'operationTime': Timestamp(1648947838, 23)}

In [67]:
collection = db['betting_data']

In [68]:
collection.insert_many(sorted_df.to_dict('records'))

***

In [125]:
%reset -fs

In [126]:
import pandas as pd
import numpy as np
import time, os
from datetime import datetime
from pymongo import MongoClient
from dotenv import load_dotenv
from sportsipy.nba.boxscore import Boxscores
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [127]:
load_dotenv()

True

In [128]:
mongo_pw = os.environ.get("MONGO_PW")

In [129]:
client = MongoClient('mongodb+srv://natedirenzo:' + mongo_pw + '@nba-db.grcwg.mongodb.net/myFirstDatabase?retryWrites=true&w=majority')


In [130]:
db = client['nba-data']

In [131]:
cursor = db['betting_data'].find()

In [132]:
mongo_df = pd.DataFrame(list(cursor)); mongo_df

,_id,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy,successful_pick
0,6248f288e3e166efae8cf592,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11.0,o,0.47,0
1,6248f288e3e166efae8cf593,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5.0,o,0.45,0
2,6248f288e3e166efae8cf594,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14.0,u,0.45,1
3,6248f288e3e166efae8cf595,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12.0,o,0.45,0
4,6248f288e3e166efae8cf596,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14.0,o,0.42,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,6248f288e3e166efae8d2eb0,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16.0,o,0.46,1
14623,6248f288e3e166efae8d2eb1,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13.0,u,0.50,0
14624,6248f288e3e166efae8d2eb2,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11.0,u,0.46,0
14625,6248f288e3e166efae8d2eb3,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0.0,u,0.46,1


In [173]:
games = Boxscores(datetime(2021, 10, 19), datetime(2022, 4, 2))


In [174]:
games = games.games

In [176]:
uri_list = []
for i in games.keys():
    for j in games[i]:
        uri_list.append(j['boxscore'])

1160